# Python, Boto3, 與 AWS S3

亞馬遜網絡服務（AWS）已成為雲計算領域的領導者。它的核心組件之一是`S3`，它是AWS提供的物件存儲服務(Object Storage Service)。憑藉其令人印象深刻的可用性和耐用性，它已成為存儲視頻，圖像和數據的標準方法。

`Boto3`是AWS的Python SDK的名稱。它允許你直接使用Python腳本來創建，更新和刪除AWS資源。

在本次的教程裡會使用地端的Minio服務器(S3-compatible)來學習如何使用Python來整合Object storage服務。

## 安裝Boto3套件

如果未來在自己本地的機器要連接到S3的服務時, 必需要先行安裝`boto3`套件。

```bash
pip install boto3
```

> 本教程的共用環境己經先行幫大家安裝好了

參考: https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/s3.html

### S3 連接參數

要連接到aws的s3服務需要以下的基本參數：

* `endpoint_url` (s3服務的網路位址)
* `aws_access_key_id` (使用者帳號)
* `aws_secret_access_key` (使用者密碼)
* `region_name` (區域編碼)

為了教學的目的, 學院在地端啟動了Minio的服務來模擬公有雲的s3服務。

以下的帳號是為了本次訓練所建(admin的權限), 例如:

* `endpoint_url` (s3服務的網路位址): **http://10.34.124.114:9500**
* `aws_access_key_id` (使用者帳號): **demo**
* `aws_secret_access_key` (使用者密碼): **demo8888**
* `region_name` (區域編碼): **us-east-1**



### Boto3的Client與Resource類別

從本質上講，Boto3所做的只是代表你去調用AWS API。對於大多數AWS服務，Boto3提供了兩種不同的方法來訪問這些抽象的API：

* **Client**: low-level 服務存取
* **Resource**: higher-level object-oriented 服務存取

你可以使用其中任何一個與S3進行交互。

要連接到low-level界面，你必須使用Boto3的`client()`構建函數。然後，輸入要連接的服務的名稱，在這種情況下為`s3`：

> API的參考文件: https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/s3.html#client

In [1]:
import boto3

s3_client = boto3.client('s3', 
                    endpoint_url='http://10.34.124.114:9500',
                    aws_access_key_id='demo',
                    aws_secret_access_key='demo8888',
                    region_name='us-east-1')

要連接到high-level界面，你將採用類似的方法，但要使用`resource()`的構建函數：

> API的參考文件: https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/s3.html#service-resource

In [2]:
import boto3

s3_resource = boto3.resource('s3',
                    endpoint_url='http://10.34.124.114:9500',
                    aws_access_key_id='demo',
                    aws_secret_access_key='demo8888',
                    region_name='us-east-1')

你已經成功連接到兩個版本的連線物件，但是現在你可能想知道，"我應該使用哪個版本"?

較low-level的`client`物件給予我們更多操控服務的能力, 但是也要求程式開發員要進行多一點的程序設計工作。

大多數`client`的SDK操作都會為你提供字典dict物件作為呼叫API後的response。如果要獲取所需的確切信息，你必須自己解析該字典dict物件。使用`resource`的SDK操作方法時，SDK會幫忙處理這樣的工作。

一般來說, `resource`連線物件是優先選用的選項。

## S3常用操作

現在你已經了解了`client`和`resource`之間的區別，讓我們開始使用它們來構建一些S3物件。

請打開瀏覽器並輸人: http:10.34.124.114:9500

![](images/minio_01_login.png)


![](images/minio_02_login.png)

### 創建一個新的Bucket

首先，你需要一個S3存儲桶。要以呼叫SDK方式來創建一個桶`bucket`，你必須首先為存儲桶選擇一個名稱。請記住，此名稱在整個AWS平台上必須唯一，因為存儲桶名稱符合DNS。如果你嘗試創建存儲桶，但另一個用戶已經聲明了相同的存儲桶名稱，則Python將會執行失敗並拋出以下錯誤：`botocore.errorfactory.BucketAlreadyExists`。

你可以通過選擇隨機名稱來增加創建存儲桶時的成功機會。在以下範例，我們使用`uuid`模塊來動態生成一個UUID4的字符串(36個字符)，我們可以添加前綴來指定每個存儲桶的用途。

In [3]:
import uuid

def create_bucket_name(bucket_prefix):
    # The generate bucket name must be between 3 and 63 chars long
    rnd_uuid = str(uuid.uuid4*())
    return f"{MY_BUCKET_PREFIX}-{rnd_uuid}"

你已經有了存儲桶名稱，但是現在還需要注意一件事：除非你使用的s3服務的區域位於美國，否則在創建存儲桶時需要明確定義`區域`。否則，Python將收到一個`IllegalLocationConstraintException`。

參考: https://docs.aws.amazon.com/AmazonRDS/latest/UserGuide/Concepts.RegionsAndAvailabilityZones.html

由於在地端, 我們使用`Minio`來做為aws s3的服務的替代, `region_name`並不是那麼重要, 但是為了符合`Boto3`的要求, 一個有效的`region_name`還是需要設定。

> API的參考文件: https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/s3.html#bucket

In [4]:
import logging
import boto3
from botocore.exceptions import ClientError

def create_bucket(bucket_name, region=None):
    """Create an S3 bucket in a specified region

    If a region is not specified, the bucket is created in the S3 default
    region (us-east-1).

    :param bucket_name: Bucket to create
    :param region: String region to create bucket in, e.g., 'us-west-2'
    :return: True if bucket created, else False
    """
    
    # Create bucket
    try:
        if region is None:
            s3_client.create_bucket(Bucket=bucket_name)
        else:
            location = {'LocationConstraint': region}
            s3_client.create_bucket(Bucket=bucket_name,
                                   CreateBucketConfiguration=location)
    except ClientError as e:
        logging.error(e)
        return False
    return True

In [5]:
MY_BUCKET_PREFIX = '8703147' # 請修改成你/妳的工號
BUCKET_01 = f"{MY_BUCKET_PREFIX}-01"
BUCKET_02 = f"{MY_BUCKET_PREFIX}-02"

# create 1st bucket
result = create_bucket(BUCKET_01)

print(f"Bucket:{BUCKET_01} created --> {result}")

# create 2nd bucket
create_bucket(BUCKET_02)

print(f"Bucket:{BUCKET_02} created --> {result}")

Bucket:8703147-01 created --> True
Bucket:8703147-02 created --> True


讓我們用Minio的browser UI來檢查是否成功創建了新的buckets。

![](images/minio_03_bucket_created.png)

現在我們有了存儲桶了。接下來，讓我們往裡頭其中添加一些Object（檔案物件)吧。

### 創建Object實例

通過使用`resource`的連線物件，你可以使用Bucket和Object的類別來與s3服務互動。

讓我們上傳一些檔案做為要儲放進`bucket`的`object`。你可以通過三種方式來上傳檔案：

* 透過一個 `Object` 實例
* 透過一個 `Bucket` 實例
* 透過 `client` 實例

#### 透過一個 `Object` 實例

假設我們想要將本地的檔案"data/hello_world.txt"上傳到s3,並設定object的key是`hello_world_01.txt`。



> API的參考文件: https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/s3.html#object

In [6]:
# setup object key
my_object_key_01 = "hello_world_01.txt"

s3_resource.Object(BUCKET_01, my_object_key_01).upload_file(Filename="data/hello_world.txt")

透過Minio的UI來檢查結果:

![](images/minio_04_object_created.png)

#### 透過一個 `Bucket` 實例

假設我們想要將本地的檔案"data/hello_world.txt"上傳到s3,並設定object的key是hello_world_02.txt。


> API的參考文件: https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/s3.html#bucket

In [7]:
# setup object key
my_object_key_02 = "hello_world_02.txt"

s3_resource.Bucket(BUCKET_01).upload_file(Key=my_object_key_02, Filename="data/hello_world.txt")

透過Minio的UI來檢查結果:

![](images/minio_05_object_created.png)

#### 透過一個 `Client` 實例

假設我們想要將本地的檔案"data/hello_world.txt"上傳到s3,並設定object的key是hello_world_03.txt。

> API的參考文件: 
> * https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/s3.html#service-resource
> * https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/s3.html#client

In [8]:
# setup object key
my_object_key_03 = "hello_world_03.txt"

s3_resource.meta.client.upload_file(
    Bucket=BUCKET_01,
    Key=my_object_key_03,
    Filename="data/hello_world.txt"
)

透過Minio的UI來檢查結果:

![](images/minio_06_object_created.png)

### 下載`Object`

要從從S3下載文件本地的檔案系統，你將按照與上傳物件時類似的步驟進行操作。但是在這種情況下，`Filename`參數將映射到本地的檔案路徑。以下範例會將文件下載到Jupyter Notebook所在的目錄：

In [9]:
# setup object key
my_object_key_01 = "hello_world_01.txt"

# download s3 object to local folder
s3_resource.Object(BUCKET_01, my_object_key_01).download_file(my_object_key_01)

透過JupyterHub的UI來檢查結果:

![](images/minio_08_object_downloaded.png)

### 在不同的`Buckets`間複製`Object`

如果你需要將文件從一個存儲桶複製到另一個存儲桶，則Boto3可以為你提供這種可能性。在以下範例中，你將使用`.copy()`將文件從第一個存儲桶複製到第二個存儲桶：

In [10]:
def copy_to_bucket(bucket_from_name, bucket_to_name, object_key):
    copy_source = {
        'Bucket': bucket_from_name,
        'Key': object_key
    }
    s3_resource.Object(bucket_to_name, object_key).copy(copy_source)
    
# setup object key
my_object_key_01 = "hello_world_01.txt"

first_bucket_name = BUCKET_01
second_bucket_name = BUCKET_02

copy_to_bucket(first_bucket_name, 
               second_bucket_name, 
               my_object_key_01)

透過Minio的UI來檢查結果:

![](images/minio_09_object_copy.png)

### 刪除一個`Object`

讓我們通過`Object`實例上調用`.delete()`從第二個存儲桶中刪除新文件：

In [11]:
# setup object key
my_object_key_01 = "hello_world_01.txt"
second_bucket_name = BUCKET_02

s3_resource.Object(second_bucket_name, my_object_key_01).delete()

{'ResponseMetadata': {'RequestId': '167E56D2F13A6F3F',
  'HostId': '',
  'HTTPStatusCode': 204,
  'HTTPHeaders': {'accept-ranges': 'bytes',
   'content-security-policy': 'block-all-mixed-content',
   'server': 'MinIO',
   'vary': 'Origin',
   'x-amz-request-id': '167E56D2F13A6F3F',
   'x-xss-protection': '1; mode=block',
   'date': 'Wed, 12 May 2021 14:06:19 GMT'},
  'RetryAttempts': 0}}

透過Minio的UI來檢查結果:

![](images/minio_10_object_deleted.png)

## 遍歷S3的Buckets與Objects

如果你需要從所有S3資源中檢索信息或對其進行操作，則`Boto3`提供了幾種迭代遍歷存儲桶和物件的方法。首先，遍歷所有已創建的存儲桶。

### 遍歷 Bucket

要遍歷你帳戶所擁有的所有buckets，你可以使用`resource`物件的`buckets`屬性來呼叫`.all()`方法，這個方法會為提供buckets實例的完整列表：

In [12]:
for bucket in s3_resource.buckets.all():
    print(bucket.name)

8703147-01
8703147-02
test


你也可以使用`client`物件來來檢索存儲桶信息，但是代碼稍稍複雜一些，因為你需要從`client`物件返回的字典dict物件中提取存儲桶訊息：

In [13]:
for bucket_dict in s3_resource.meta.client.list_buckets().get('Buckets'):
    print(bucket_dict['Name'])

8703147-01
8703147-02
test


你已經了解瞭如何遍歷帳戶中的buckets。在接下來的部分中，你將選擇一個bucket，並查看其中a儲放的objects。

### 遍歷 Object

如果要列出bucket中的所有objects，則以下程式碼將為你生成一個itrator：

In [14]:
my_bucket_01 = s3_resource.Bucket(BUCKET_01)

for obj in my_bucket_01.objects.all():
    print(obj.key)

hello_world_01.txt
hello_world_02.txt
hello_world_03.txt


obj變數是一個`ObjectSummary`物件。這是對儲放在s3裡頭object的一種輕量級的表示。`ObjectSummary`物件不支持`object`具有的所有屬性。

In [15]:
for obj in my_bucket_01.objects.all():
    print(type(obj))
    s3_obj = obj.Object()
    print(type(s3_obj))
    print(obj.key, obj.storage_class, obj.last_modified,
          s3_obj.version_id, s3_obj.metadata)

<class 'boto3.resources.factory.s3.ObjectSummary'>
<class 'boto3.resources.factory.s3.Object'>
hello_world_01.txt STANDARD 2021-05-12 14:06:19.749000+00:00 None {}
<class 'boto3.resources.factory.s3.ObjectSummary'>
<class 'boto3.resources.factory.s3.Object'>
hello_world_02.txt STANDARD 2021-05-12 14:06:19.781000+00:00 None {}
<class 'boto3.resources.factory.s3.ObjectSummary'>
<class 'boto3.resources.factory.s3.Object'>
hello_world_03.txt STANDARD 2021-05-12 14:06:19.801000+00:00 None {}


現在，你可以迭代地對buckets和object報行操作。接著，你還需要了解一件事：如何刪除在本教程中創建的所有`resoures`。

## 刪除Buckets與Objects

要刪除所有已創建的buckets和objects，必須首先確保bucket中沒有object。

### 刪除非空的Bucket

為了能夠刪除bucket，你必須首先刪除bucket中的每個objects，否則會引發`BucketNotEmpty`異常。當擁有版本化bucket時，你還需要刪除每個objects及其所有版本。

如果你發現可以自動執行此操作的`LifeCycle`規則不適合你的需求，則可以通過以下方式以寫程式的方式來刪除objects：

In [16]:
my_bucket_01 = s3_resource.Bucket(BUCKET_01)

my_bucket_01.objects.delete()

[{'ResponseMetadata': {'RequestId': '167E56D2F7708D35',
   'HostId': '',
   'HTTPStatusCode': 200,
   'HTTPHeaders': {'accept-ranges': 'bytes',
    'content-length': '260',
    'content-security-policy': 'block-all-mixed-content',
    'content-type': 'application/xml',
    'server': 'MinIO',
    'vary': 'Origin',
    'x-amz-request-id': '167E56D2F7708D35',
    'x-xss-protection': '1; mode=block',
    'date': 'Wed, 12 May 2021 14:06:19 GMT'},
   'RetryAttempts': 0},
  'Deleted': [{'Key': 'hello_world_01.txt'},
   {'Key': 'hello_world_02.txt'},
   {'Key': 'hello_world_03.txt'}]}]

### 刪除Bucket

最後，你可以在`bucket`實例上使用`.delete()`方法來刪除bucket：

In [17]:
my_bucket_01 = s3_resource.Bucket(BUCKET_01)

s3_resource.Bucket(first_bucket_name).delete()

{'ResponseMetadata': {'RequestId': '167E56D2F8236F28',
  'HostId': '',
  'HTTPStatusCode': 204,
  'HTTPHeaders': {'accept-ranges': 'bytes',
   'content-security-policy': 'block-all-mixed-content',
   'server': 'MinIO',
   'vary': 'Origin',
   'x-amz-request-id': '167E56D2F8236F28',
   'x-xss-protection': '1; mode=block',
   'date': 'Wed, 12 May 2021 14:06:20 GMT'},
  'RetryAttempts': 0}}

透過Minio的UI來檢查結果:

![](images/minio_11_bucket_deleted.png)

如果需要，你可以使用`client`的物件來刪除bucket：

In [18]:
second_bucket_name = BUCKET_02

s3_resource.meta.client.delete_bucket(Bucket=second_bucket_name)

{'ResponseMetadata': {'RequestId': '167E56D2F89A4B86',
  'HostId': '',
  'HTTPStatusCode': 204,
  'HTTPHeaders': {'accept-ranges': 'bytes',
   'content-security-policy': 'block-all-mixed-content',
   'server': 'MinIO',
   'vary': 'Origin',
   'x-amz-request-id': '167E56D2F89A4B86',
   'x-xss-protection': '1; mode=block',
   'date': 'Wed, 12 May 2021 14:06:20 GMT'},
  'RetryAttempts': 0}}

透過Minio的UI來檢查結果:

![](images/minio_12_bucket_deleted.png)

# 使用Pandas在Amazon S3中讀寫文件

透過boto3來與pandas進行整合。

參考: https://towardsdatascience.com/reading-and-writing-files-from-to-amazon-s3-with-pandas-ccaf90bfe86c

![](images/minio_13_pandas.png)

## `tempfile` 模組產生暫存檔案、目錄

介紹如何使用 Python 的 `tempfile` 模組自動建立暫存檔案或目錄，存放暫時性的資料。

### 建立暫存檔案

`TemporaryFile()`方法可以用來建立暫存用的檔案，預設會以**二進位**的方式開啟，我們可以將暫存資料寫入檔案中，需要時再讀取出來，**而在檔案關閉之後，該暫存檔就會自動被刪除**。

In [19]:
# 引入 tempfile 模組
import tempfile

# 建立並開啟暫存檔案（二進位）
fp = tempfile.TemporaryFile()

# 寫入二進位資料
fp.write(b'Hello, world.')

# 將檔案讀寫位置（offset）設定為最開頭
fp.seek(0)

# 讀取檔案內容
content = fp.read()

# 輸出結果
print(content)

# 關閉檔案（自動刪除檔案）
fp.close()

b'Hello, world.'


如果想要儲存的資料是文字，可以改用文字模式開啟暫存檔：

In [20]:
import tempfile

# 建立並開啟暫存檔案（文字檔）
fp = tempfile.TemporaryFile(mode = 'w+')

# 寫入文字資料
fp.write('Hello, world.')

fp.seek(0)
content = fp.read()
print(content)
fp.close()

Hello, world.


若搭配 `with` 使用，可讓語法更簡潔：

In [21]:
import tempfile

with tempfile.TemporaryFile('w+') as fp:
    fp.write('Hello, world.')
    fp.seek(0)
    content = fp.read()
    print(content)
    
# 自動刪除暫存檔

Hello, world.


### 建立具名暫存檔案

使用 `TemporaryFile()` 所建立的暫存檔是匿名的，如果需要取得檔案名稱，可以改用 `NamedTemporaryFile()`：

In [22]:
import tempfile

with tempfile.NamedTemporaryFile('w+') as fp:
    print('暫存檔：', fp.name)
    # ... processing ....
    
# 自動刪除暫存檔

暫存檔： /tmp/tmpu7nhogoe


具名的暫存檔案一樣會在關閉時自動被刪除，如果希望自己管理暫存檔案的刪除動作，可以加上 `delete=False` 參數：

In [23]:
import tempfile
import os

with tempfile.NamedTemporaryFile('w+', delete=False) as fp:
    tmpFilename = fp.name
    # ...

# 自行刪除暫存檔
os.unlink(tmpFilename)

### 建立暫存目錄

若需要建立一個暫存目錄，存放各種暫存檔案，可以使用 TemporaryDirectory() 函數，在暫存目錄中可以存放任意數量的暫存檔，使用完畢之後，該暫存目錄（連同裡面所有檔案）會自動被刪除。

In [24]:
import tempfile

with tempfile.TemporaryDirectory() as dirname:
    print('暫存目錄：', dirname)
    # ...

# 自動刪除暫存目錄與所有內容

暫存目錄： /tmp/tmp1s4tw33c


## 將`data frame`寫出成CSV檔案並上傳至S3

透過`tempfile`模組產生暫存檔案來讓pandas的`to_csv()`方法產生出目標結構的資料檔案之後, 接著使用boto3的API來上傳檔案到s3的bucket中。

> 如果bucket不存在時, 以下的程式碼會失敗

In [25]:
import io
import os
import pandas as pd
import boto3
import tempfile

s3_resource = boto3.resource('s3',
                    endpoint_url='http://10.34.124.114:9500',
                    aws_access_key_id='demo',
                    aws_secret_access_key='demo8888',
                    region_name='us-east-1')

# 構建dataframe
books_df = pd.DataFrame(
    data={"Title": ["Book I", "Book II", "Book III"], "Price": [56.6, 59.87, 74.54]},
    columns=["Title", "Price"],
)

print(books_df.info())
print(books_df.head())

my_bucket_name = 'test'
my_object_key = 'test.csv'

with tempfile.NamedTemporaryFile('w+') as fp:
    temp_filename = fp.name # 暫存檔案名
    books_df.to_csv(temp_filename, index=False) # 快捷鍵 SHIFT + TAB
    try:
        s3_resource.Object(my_bucket_name, my_object_key).upload_file(Filename=temp_filename)
        print('Upload file to s3 success!')
    except ClientError as e:
        printt('Upload file to s3 fail:{e}')
    
# 自行刪除暫存檔

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3 entries, 0 to 2
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Title   3 non-null      object 
 1   Price   3 non-null      float64
dtypes: float64(1), object(1)
memory usage: 176.0+ bytes
None
      Title  Price
0    Book I  56.60
1   Book II  59.87
2  Book III  74.54
Upload file to s3 success!


透過Minio的UI來檢查結果:

![](images/minio_14_pandas_s3.png)

下載`test.csv`並使用Excel打開它。

![](images/minio_15_pandas_s3.png)


## 讀取S3上的CSV的物件成為`data frame`

使用`boto3`的API從s3的bucket來下載檔案物件本地的檔案(透過`tempfile`模組產生暫存檔案）。接著讓pandas的`read_csv()`方法把數據轉換成DataFrame物件。

In [26]:
import io
import os
import pandas as pd
import boto3
import tempfile

s3_resource = boto3.resource('s3',
                    endpoint_url='http://10.34.124.114:9500',
                    aws_access_key_id='demo',
                    aws_secret_access_key='demo8888',
                    region_name='us-east-1')




my_bucket_name = 'test'
my_object_key = 'test.csv'

with tempfile.NamedTemporaryFile('w+') as fp:
    temp_filename = fp.name # 暫存檔案名    
    # download s3 object to local folder
    try:
        s3_resource.Object(my_bucket_name, my_object_key).download_file(temp_filename)
        print('Download s3 object to local success!')
    except ClientError as e:
        printt('Download s3 object to local fail:{e}')
    

    # read CSV into DataFrame
    book_df2 = pd.read_csv(temp_filename)
    
    print(books_df.info())
    print(books_df.head())
    
    
# 自行刪除暫存檔

Download s3 object to local success!
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3 entries, 0 to 2
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Title   3 non-null      object 
 1   Price   3 non-null      float64
dtypes: float64(1), object(1)
memory usage: 176.0+ bytes
None
      Title  Price
0    Book I  56.60
1   Book II  59.87
2  Book III  74.54
